In [1]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd
import warnings
import nltk
import spacy
import en_core_web_md
import matplotlib.pyplot as plt
import plotly.express as px
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
nlp = spacy.load('en_core_web_md')

2023-05-04 10:24:58.163342: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 10:24:58.323920: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-04 10:24:58.905592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-04 10:24:58.905676: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
df = pd.read_csv('./Data/genData/text_sentences_words.csv') # this file has the texts with punctuation!

In [3]:
# remove [stuff] in between square brackets
def remove_bracket(text):
    return re.sub('(\[[^w]*\]\s)', '',text)
df['text'] = df['text'].apply(remove_bracket)

In [4]:
df.head(2)

,text,filepath,date,source,sentences,words,num_sents,num_words,word_set,num_unique_words
0,"Good afternoon, everybody. One year ago this m...",./Data/speeches/2014-07-01-Immigration.txt,2014-07-01,oba,"['Good afternoon, everybody.', 'One year ago t...","['Good', 'afternoon', ',', 'everybody.', 'One'...",112,2095,"{'strengthen', 'barbecue', 'Republican', 'righ...",725
1,"Good morning, everybody. I want to take just a...",./Data/speeches/2009-12-25-UnderwearBomber.txt,2009-12-25,oba,"['Good morning, everybody.', ""I want to take j...","['Good', 'morning', ',', 'everybody.', 'I', 'w...",53,1166,"{'seek', 'destroy', 'season.', 'international....",501


In [5]:
#df[enc_col[j]][i] = list(nlp(df[sources[j]][i]).vector) <- from textPCASimilarity.ipynb
def get_encodings(text):
    return list(nlp(text).vector)
df['enc'] = df['text'].apply(get_encodings)

In [6]:
pca_data = pd.DataFrame(df['enc'].to_list())

In [7]:
pca = PCA(n_components = 2)
pcafit = pca.fit_transform(df['enc'].to_list())
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}
labels["color"] = "source"

<A HREF="https://plotly.com/python/discrete-color/#color-sequences-in-plotly-express">Plotly discrete color palettes/sequences</A><BR><A HREF="https://plotly.com/python/plotly-express/">Plotly Express docs</A>

In [11]:
fig = px.scatter(pcafit, 
                 x=0, 
                 y=1, 
                 color=df['source'],
                 hover_name=df['date'],
                 labels = labels,
                 title  = "PCA using Spacy encodings",
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

In [12]:
fig.write_image("./plots/PCA_using_Spacy_encodings_oba_nyt_wsj.png", format='png', engine='kaleido')

In [9]:
def highlight_max(x):
    return ['font-weight: bold' if abs(v) > 0.15 else ''
                for v in x]
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=pca_data.columns)
loadings.sort_values(by=['PC1'], inplace=True)
loadings.style.apply(highlight_max)

,PC1,PC2
122,-0.150859,0.064919
120,-0.146986,-0.048783
17,-0.141532,0.052144
230,-0.138345,0.124009
187,-0.135880,0.078006
240,-0.131275,-0.028514
275,-0.121136,-0.089977
23,-0.120629,0.089298
93,-0.118278,0.019732
19,-0.114150,-0.123887


In [27]:
dists = pd.DataFrame(euclidean_distances(pcafit, pcafit))

In [28]:
max_dist = dists.max()[dists.max() == dists.max(axis=1).max()].index 
dists.max()[dists.max() == dists.max(axis=1).max()]

15     21.916203
148    21.916203
dtype: float64

In [29]:
df.iloc[max_dist[0]]

text                Well you know, I have to tell you, as I've sai...
filepath                    ./Data/speeches/2012-05-10-SameSexMarriage.txt
date                                                       2012-05-10
source                                                            oba
sentences           ["Well you know, I have to tell you, as I've s...
words               ['Well', 'you', 'know', ',', 'I', 'have', 'to'...
num_sents                                                          77
num_words                                                        1710
word_set            {'strengthen', 'gone', 'strongly', 'voted', 's...
num_unique_words                                                  526
enc                 [-1.5855007, 1.6168158, -3.3720534, -0.7653962...
Name: 15, dtype: object

In [30]:
df.iloc[max_dist[1]]

text                The Intel Corporation announced plans on Frida...
filepath                           ./Data/NYTimes/2011-02-19-IntelCorp.txt
date                                                       2011-02-19
source                                                            nyt
sentences           ['The Intel Corporation announced plans on Fri...
words               ['The', 'Intel', 'Corporation', 'announced', '...
num_sents                                                           4
num_words                                                          87
word_set            {'5', 'The', 'by', 'President', 'research', 'a...
num_unique_words                                                   63
enc                 [-2.3413625, -0.080605224, -2.1773891, 1.05917...
Name: 148, dtype: object

<A HREF="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Pandas Cheat Sheet</A>

In [31]:
date1='2012-07-21'
source1='oba'
df.query('date==@date1 and source==@source1')

,text,filepath,date,source,sentences,words,num_sents,num_words,word_set,num_unique_words,enc
37,Let me first of all say how grateful I am for ...,./Data/speeches/2012-07-21-AuroraShooting.txt,2012-07-21,oba,"[""Let me first of all say how grateful I am fo...","['Let', 'me', 'first', 'of', 'all', 'say', 'ho...",45,887,"{'Lord', 'precious.', 'lives.', 'gunman', 'mov...",369,"[-1.3268358, 1.1527715, -3.616854, -0.9435685,..."


In [32]:
date2='2010-07-26'
source2='wsj'
df.query('date==@date2 and source==@source2')

,text,filepath,date,source,sentences,words,num_sents,num_words,word_set,num_unique_words,enc
264,Thousands of secret military documents were re...,./Data/WSJ/2010-07-26-Wikileaks.txt,2010-07-26,wsj,['Thousands of secret military documents were ...,"['Thousands', 'of', 'secret', 'military', 'doc...",39,1043,"{'Officials', 'Sunday', 'period', 'increasing'...",474,"[-2.369939, 0.24754976, -1.3718952, 0.9974633,..."


In [33]:
dists.iloc[df.query('date==@date1 and source==@source1').index,
           df.query('date==@date2 and source==@source2').index]

,264
37,19.39022
